References:
- [Hyperparameter tuning in XGBoost](https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f)

# Instagram Like Prediction @310ai Competition - Modeling, Visualization & Evaluation

This notebook is a continuation of ***Data*** notebook. In this part of project we will train the predictive model, analyze it, visualize its output and evaluate it. The reason for this separation, is to make the notebooks easier to understand and less dependent on each other, especially for this case since the Instagram showed that it reguralry changing its api and bot mitigation methods. Multiple stages of this project have checkpoints, thus the notebook will run without issues.

As we pointed out in the previous notebook, we will use **XGBoost 1.7** as the algorithm. So let's without further ado, dive into it.

In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

First thing first, we must load the data, also we have some identifying features, called shortcode, username and id, which will not affect the accuracy of the model, thus we can drop these features all together.

In [3]:
df = pd.read_csv('Data/main_dataset.csv')
df.drop(columns=['Unnamed: 0', 'shortcode', 'username', 'id'], inplace=True)
df.head()

,post_type,like,comment,object_1,object_2,object_3,object_4,object_5,object_6,object,...,reel_count,reel_avg_view,reel_avg_comment,reel_avg_like,reel_avg_duration,reel_frequency,media_count,media_avg_comment,media_avg_like,media_frequency
0,GraphSidecar,363269,6844,No Object,No Object,No Object,No Object,No Object,No Object,palace,...,1256,"9,406,907.25","13,544.50","634,500.67",92.47,18.55,7393,"8,635.17","410,027.25",1.09
1,GraphSidecar,546578,11354,No Object,No Object,No Object,No Object,No Object,No Object,"bikini, two-piece",...,1256,"9,406,907.25","13,544.50","634,500.67",92.47,18.55,7393,"8,635.17","410,027.25",1.09
2,GraphImage,734124,11937,No Object,No Object,No Object,No Object,No Object,No Object,"balance_beam, beam",...,1256,"9,406,907.25","13,544.50","634,500.67",92.47,18.55,7393,"8,635.17","410,027.25",1.09
3,GraphSidecar,7720568,42249,2 people,people playing football,people playing soccer,ball,No Object,No Object,basketball,...,387,"18,249,237.58","28,318.83","3,607,849.92",32.62,17.04,3475,"52,543.17","7,980,593.08",2.20
4,GraphImage,14451079,103278,1 person,baby,No Object,No Object,No Object,No Object,"crib, cot",...,387,"18,249,237.58","28,318.83","3,607,849.92",32.62,17.04,3475,"52,543.17","7,980,593.08",2.20


Now we can separate independent and dependent variables. Usually we cal independent variables X and dependent variables y.

In [4]:
# remove objects from instagram detection algorithm and only use efficient net algorithm
exclude_columns = [
    'like',
    'object_1',
    'object_2',
    'object_3',
    'object_4',
    'object_5',
    'object_6',
]
X, y = df.loc[:, ~df.columns.isin(exclude_columns)].copy(), df[['like']].copy()

As we said in the previous notebook, XGBoost since the version of 1.7 is capable of working with categorical variables, but before casting the categorical variables through the model, we must change the type of those features to `categorical` in pandas.

In [5]:
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

Now we can split the data into training and validation. We will use the 25% of data as validation set.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=69)

dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

### What architecture and pre trained models -if any- you used?

In this cell we will define hyperparameters value for model. Have in mind that since we will perform a cross-validation, parameters are actually list of parameters to find the best one among their combination.

In [7]:
# creating a baseline model
mean_train = np.mean(np.asarray(y_train))
baseline_predictions = np.ones(np.asarray(y_test).shape) * mean_train
mae_baseline = mean_absolute_error(np.asarray(y_test), baseline_predictions)
print(f'Baseline MAE: {mae_baseline:.2f}')

Baseline MAE: 1832111.19


In [13]:
params = {
    'max_depth': 6,             # maximum depth of a tree
    'min_child_weight': 1,      # minimum sum of weights of all observations required in a child
    'eta': .25,                 # learning rate
    'subsample': 1,             # fraction of observation to be random samples for each tree
}
num_boost_rounds = 1500

Tunning `max_depth`, `min_child_weight`, `eta` and `subsample` hyperparameters.

In [9]:
grid_search_params = [
    (max_depth, min_child_weight, eta, subsample)
    for max_depth in range(5, 14)
    for min_child_weight in range(3, 10)
    for eta in [.3, .2, .1, .05, .01, .005]
    for subsample in [i/10. for i in range(7, 11)]
]

min_mae = float('Inf')
best_params = None
for max_depth, min_child_weight, eta, subsample in grid_search_params:
    print(f'CV with max_depth: {max_depth}, min_child_weight: {min_child_weight}, eta: {eta}, subsample: {subsample}')

    # updating parameters dictionary
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    params['eta'] = eta
    params['subsample'] = subsample

    # performing cv
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_rounds,
        seed=69,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10,
    )

    # updating the best mae
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print(f'\tMAE: {mean_mae:,.2f}, rounds: {boost_rounds}')
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth, min_child_weight, eta, subsample)

# best hyperparameters combination
print('Best hyperparameters:')
print(f'\tmax_depth: {max_depth}')
print(f'\tmin_child_weight: {min_child_weight}')
print(f'\teta: {eta}')
print(f'\tsubsample: {subsample}')
print(f'\tmae: {min_mae}')


CV with max_depth: 5, min_child_weight: 3, eta: 0.3, subsample: 0.7
	MAE: 430,303.75, rounds: 25
CV with max_depth: 5, min_child_weight: 3, eta: 0.3, subsample: 0.8
	MAE: 442,774.24, rounds: 25
CV with max_depth: 5, min_child_weight: 3, eta: 0.3, subsample: 0.9
	MAE: 441,058.63, rounds: 16
CV with max_depth: 5, min_child_weight: 3, eta: 0.3, subsample: 1.0
	MAE: 420,741.64, rounds: 15
CV with max_depth: 5, min_child_weight: 3, eta: 0.2, subsample: 0.7
	MAE: 424,710.19, rounds: 25
CV with max_depth: 5, min_child_weight: 3, eta: 0.2, subsample: 0.8
	MAE: 416,182.94, rounds: 27
CV with max_depth: 5, min_child_weight: 3, eta: 0.2, subsample: 0.9
	MAE: 434,100.67, rounds: 21
CV with max_depth: 5, min_child_weight: 3, eta: 0.2, subsample: 1.0
	MAE: 414,795.39, rounds: 14
CV with max_depth: 5, min_child_weight: 3, eta: 0.1, subsample: 0.7
	MAE: 429,602.15, rounds: 32
CV with max_depth: 5, min_child_weight: 3, eta: 0.1, subsample: 0.8
	MAE: 420,045.38, rounds: 34
CV with max_depth: 5, min_chil

As we can see from the result of cross validation, the best hyperparameters for the model is:
- **max_depth**: 13
- **min_child_weight**: 9
- **eta**: 0.005
- **subsample**: 1

now lets train a model based on these hyperparameters and test it with test dataset.

In [24]:
params = {
    'max_depth': 13,            # maximum depth of a tree
    'min_child_weight': 9,      # minimum sum of weights of all observations required in a child
    'eta': .005,                # learning rate
    'subsample': 1,             # fraction of observation to be random samples for each tree
    'eval_metric': 'mae',
}
num_boost_rounds = 1500

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_rounds,
    evals=[(dtest, 'test')],
    early_stopping_rounds=10,
)

[0]	test-mae:1524358.51899
[1]	test-mae:1517111.39984
[2]	test-mae:1509898.41752
[3]	test-mae:1502718.39702
[4]	test-mae:1495572.90186
[5]	test-mae:1488461.21687
[6]	test-mae:1481381.15923
[7]	test-mae:1474335.46814
[8]	test-mae:1467434.20652
[9]	test-mae:1460298.81124
[10]	test-mae:1453350.41549
[11]	test-mae:1446457.17773
[12]	test-mae:1439405.22968
[13]	test-mae:1432543.63640
[14]	test-mae:1425663.57670
[15]	test-mae:1418481.98502
[16]	test-mae:1411332.98075
[17]	test-mae:1404241.99510
[18]	test-mae:1397159.93716
[19]	test-mae:1390454.40136
[20]	test-mae:1383438.56771
[21]	test-mae:1376505.63758
[22]	test-mae:1369951.89796
[23]	test-mae:1363098.79189
[24]	test-mae:1356268.57180
[25]	test-mae:1349441.04898
[26]	test-mae:1342696.67784
[27]	test-mae:1335972.25355
[28]	test-mae:1329599.27475
[29]	test-mae:1322975.75861
[30]	test-mae:1316676.92849
[31]	test-mae:1310066.56990
[32]	test-mae:1303405.41055
[33]	test-mae:1297183.09701
[34]	test-mae:1290946.08831
[35]	test-mae:1284782.85183
[3

In [26]:
print(f'Best MAE: {model.best_score:,.2f} in {model.best_iteration + 1} round.')

Best MAE: 298,372.70 in 528 round.


Now we can train the best model for the purpose, now we have use all of the data to train a final model and deply it.

In [29]:
num_boost_rounds = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round= num_boost_rounds,
    evals=[(dtest, 'Test')]
)

[0]	Test-mae:1524358.51899
[1]	Test-mae:1517111.39984
[2]	Test-mae:1509898.41752
[3]	Test-mae:1502718.39702
[4]	Test-mae:1495572.90186
[5]	Test-mae:1488461.21687
[6]	Test-mae:1481381.15923
[7]	Test-mae:1474335.46814
[8]	Test-mae:1467434.20652
[9]	Test-mae:1460298.81124
[10]	Test-mae:1453350.41549
[11]	Test-mae:1446457.17773
[12]	Test-mae:1439405.22968
[13]	Test-mae:1432543.63640
[14]	Test-mae:1425663.57670
[15]	Test-mae:1418481.98502
[16]	Test-mae:1411332.98075
[17]	Test-mae:1404241.99510
[18]	Test-mae:1397159.93716
[19]	Test-mae:1390454.40136
[20]	Test-mae:1383438.56771
[21]	Test-mae:1376505.63758
[22]	Test-mae:1369951.89796
[23]	Test-mae:1363098.79189
[24]	Test-mae:1356268.57180
[25]	Test-mae:1349441.04898
[26]	Test-mae:1342696.67784
[27]	Test-mae:1335972.25355
[28]	Test-mae:1329599.27475
[29]	Test-mae:1322975.75861
[30]	Test-mae:1316676.92849
[31]	Test-mae:1310066.56990
[32]	Test-mae:1303405.41055
[33]	Test-mae:1297183.09701
[34]	Test-mae:1290946.08831
[35]	Test-mae:1284782.85183
[3

In [31]:
mean_absolute_error(best_model.predict(dtest), y_test)

298372.7003655991

In [32]:
best_model.save_model('Models/xgb.pth')

TODO:
- now must prepare a pipeline, and create deploy it as an api to be used by users.